In [ ]:
import os

import scanpy as sc

from gears import GEARS, PertData

In [ ]:
# Load "norman" data.
pert_data = PertData(data_path="./data")
pert_data.load(data_name="norman")

In [ ]:
# Load "ReplogleNorman2020_E7" data.
# - This data needs to be generated by executing
#   "ReplogleNorman2020_E7/ReplogleNorman2020_E7.ipynb".
# - It must contain a "gene_name" column in adata.var, and two columns "condition",
#   "cell_type" in adata.obs.
dataset_name = "ReplogleNorman2020_E7"
adata = sc.read_h5ad(filename=os.path.join(f"{dataset_name}", "adata.h5ad"))
pert_data.new_data_process(dataset_name=dataset_name, adata=adata)
pert_data.load(data_path=f"./data/{dataset_name}")

In [ ]:
# Specify data split.
pert_data.prepare_split(split="simulation", seed=42)

# Get dataloader.
pert_data.get_dataloader(batch_size=32, test_batch_size=128)

In [ ]:
# Set up and train GEARS model.
gears_model = GEARS(pert_data=pert_data)
gears_model.model_initialize()
gears_model.train()

In [ ]:
# # Save or load a model.
# gears_model.save_model(path="gears")
# gears_model.load_pretrained(path="gears")

In [ ]:
# # Predict.
# predictions, _ = gears_model.predict(pert_list=[["CBL", "CNN1"], ["FEV"]])